In [1]:
import os
import pandas as pd
import numpy as np
import cv2
import shutil

In [2]:
def process_heatmap(heatmap_path, threshold=128):
    # Load the heatmap
    heatmap = cv2.imread(heatmap_path, cv2.IMREAD_GRAYSCALE)

    # Threshold the heatmap to find the "hot" areas
    _, binary_heatmap = cv2.threshold(heatmap, threshold, 255, cv2.THRESH_BINARY)

    # Find contours in the binary heatmap
    contours, _ = cv2.findContours(binary_heatmap, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Assuming the largest contour corresponds to the ball
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Convert bounding box coordinates to YOLO format
        img_height, img_width = heatmap.shape
        x_center = (x + w / 2) / img_width
        y_center = (y + h / 2) / img_height
        width = w / img_width
        height = h / img_height

        return [0, x_center, y_center, width, height]  # Class ID is assumed to be 0


In [26]:
def create_yolo_annotations(heatmaps_folder):
    for filename in os.listdir(heatmaps_folder):        
        if filename.endswith("_heatmap.jpg"):
            heatmap_path = os.path.abspath(os.path.join(heatmaps_folder, filename))
            bbox = process_heatmap(heatmap_path)

            if bbox:
                # Prepare YOLO annotation line
                yolo_format = ' '.join(map(str, bbox))

                # Write the annotation to a file
                annotation_filename = filename.replace('_heatmap.jpg', '.txt')
                annot_folder = "/".join(heatmaps_folder.split("/")[:-1])
                with open(os.path.join(annot_folder, annotation_filename), 'w') as file:
                    file.write(yolo_format + '\n')


In [4]:
def create_heatmap(x, y, width, height, sigma=25):
    xs = np.linspace(0, width, width)
    ys = np.linspace(0, height, height)
    xs, ys = np.meshgrid(xs, ys)

    # Create a Gaussian centered at (x, y)
    heatmap = np.exp(-((xs - x)**2 + (ys - y)**2) / (2 * sigma**2))

    return heatmap



In [5]:

def save_heatmaps_for_clip(clip_path, output_path, image_shape, x, y):
    game, clip, clip_name = clip_path.split("/")[-3:]
    heatmap_filename = game + "_" + clip + "_" + clip_name.split(".")[0] + '_heatmap.jpg'
    
    image_path = os.path.join("/".join(output_path.split("/")[:-1]), game + "_" + clip + "_" + clip_name)
    
    if not os.path.exists(os.path.join(output_path, heatmap_filename)):
        heatmap = create_heatmap(x, y, image_shape[1], image_shape[0])

        # Normalize, convert to uint8, and save the heatmap
        heatmap_normalized = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX)
        heatmap_normalized = np.uint8(heatmap_normalized)
        
        shutil.copy(clip_path, image_path)
        cv2.imwrite(os.path.join(output_path, heatmap_filename), heatmap_normalized)


In [15]:
base_dataset_path = "/kaggle/input/gamedataset/Dataset"
base_path = "/kaggle/working/"

os.makedirs("/kaggle/working/train/heatmaps", exist_ok=True)
os.makedirs("/kaggle/working/test/heatmaps", exist_ok=True)
os.makedirs("/kaggle/working/val/heatmaps", exist_ok=True)

heatmaps = []
status = []
image_shape = (720, 1280)  # Assuming 720p resolution for the heatmaps
count = 0
for i, game_folder in enumerate(os.listdir(base_dataset_path)):
    game_path = os.path.join(base_dataset_path, game_folder)
    
    if(game_folder not in ("Readme.docx", ".DS_Store")):
        count += 1
        for clip_folder in os.listdir(game_path):
            if(clip_folder !=  ".DS_Store"):
                clip_path = os.path.join(game_path, clip_folder)
                labels = pd.read_csv(os.path.join(clip_path, 'Label.csv'))

                for _, row in labels.iterrows():
                    x, y = row['x-coordinate'], row['y-coordinate']

                    if row['visibility'] > 0:
                        image_path = os.path.join(clip_path, row['file name'])
                        if (count < 3):
                            destination_folder = os.path.join(base_path, "train", "heatmaps")
                        elif (count >= 3 and count < 4):
                            destination_folder = os.path.join(base_path, "val", "heatmaps")
                        elif (count >= 4 and count < 5):
                            destination_folder = os.path.join(base_path, "test", "heatmaps")
                        else:
                            break
                        save_heatmaps_for_clip(image_path, destination_folder, image_shape, x, y)


In [14]:
shutil.rmtree('/kaggle/working/train')
shutil.rmtree('/kaggle/working/test')
shutil.rmtree('/kaggle/working/val')


In [45]:
%cd /kaggle/working
from IPython.display import FileLinks

#!zip -r file.zip /kaggle/working
FileLink(r'file.zip')


/kaggle/working


/kaggle/working/file.zip

In [27]:

import warnings

warnings.filterwarnings("ignore")

create_yolo_annotations("/kaggle/working/train/heatmaps")
create_yolo_annotations("/kaggle/working/test/heatmaps")
create_yolo_annotations("/kaggle/working/val/heatmaps")

warnings.filterwarnings("default")


In [28]:
!git clone https://github.com/ultralytics/yolov5.git


Cloning into 'yolov5'...
remote: Enumerating objects: 16094, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 16094 (delta 0), reused 4 (delta 0), pack-reused 16089
Receiving objects: 100% (16094/16094), 14.82 MiB | 22.55 MiB/s, done.
Resolving deltas: 100% (11023/11023), done.


In [29]:
%cd /kaggle/working/yolov5
!pwd  # Should display /content/yolov5


/kaggle/working/yolov5
/kaggle/working/yolov5


In [30]:
!pip install -r requirements.txt


In [57]:
!pip install pyyaml

In [31]:
import yaml

# Your data to be written to the YAML file
data = {
    'train': '/kaggle/working/train',
    'val': '/kaggle/working/val',
    'nc': 1,
    'names': ['ball']
}

# Specify the path for the YAML file
yaml_file_path = '/kaggle/working/dataset.yaml'

# Write data to the YAML file
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(data, yaml_file, default_flow_style=False)

print(f"YAML file '{yaml_file_path}' created successfully.")


YAML file '/kaggle/working/dataset.yaml' created successfully.


In [21]:
import os
import shutil

def copy_txt_files(source_folder, destination_folder):
    for filename in os.listdir(source_folder):
        if filename.endswith(".txt"):
            source_path = os.path.join(source_folder, filename)
            destination_path = os.path.join(destination_folder, filename)
            shutil.copy(source_path, destination_path)

source_folder = "/kaggle/working/train/heatmaps"
destination_folder = "/kaggle/working/train"
copy_txt_files(source_folder, destination_folder)

source_folder = "/kaggle/working/test/heatmaps"
destination_folder = "/kaggle/working/test"
copy_txt_files(source_folder, destination_folder)

source_folder = "/kaggle/working/val/heatmaps"
destination_folder = "/kaggle/working/val"
copy_txt_files(source_folder, destination_folder)


In [32]:
!python train.py --img 640 --batch 16 --epochs 50 --data '/kaggle/working/dataset.yaml' --weights yolov5s.pt --cache


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5s.pt, cfg=, data=/kaggle/working/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github

In [35]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source /kaggle/working/test/

#game2_Clip5_0129.jpg


detect: weights=['runs/train/exp/weights/best.pt'], source=/kaggle/working/test/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: Ultralytics requirement ['Pillow>=10.0.1'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 11.0s, installed 1 package: ['Pillow>=10.0.1']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 v7.0-249-gf400bba Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/2219 /kaggle/working/test/game2_Clip1_0000.jpg: 384x640 (no detections),

In [37]:
%cd runs/detect/exp2

/kaggle/working/yolov5/runs/detect/exp2


In [39]:
%cd /kaggle/working/yolov5


/kaggle/working/yolov5


In [40]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source /kaggle/input/sampletennisvideo/sample1.mp4


detect: weights=['runs/train/exp/weights/best.pt'], source=/kaggle/input/sampletennisvideo/sample1.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: Ultralytics requirement ['Pillow>=10.0.1'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 11.0s, installed 1 package: ['Pillow>=10.0.1']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 v7.0-249-gf400bba Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/638) /kaggle/input/sampletennisvideo/sample1.